<a href="https://colab.research.google.com/github/alehunter/CAP4770/blob/main/projects/rAlejandroHunter_bach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before Starting...
Everytime Runtime-> Run Before or Run All are executed, new zipfiles and csv's are added to the notebook. 
Let's make sure we clear the directory everytime we run from the beginning 

In [124]:
!ls

bach.csv  bach.zip


In [125]:
!rm * -Rf

In [126]:
!ls

<h1>__PREDICTING CHORDS FROM BACH COMPOSITIONS__</h1>

Our goal in this project is to predict the chord being played in a specific "event" given a row of notes in a dataframe of many notes from many different compositions.

Our first step should be to see what the data looks like.

In [127]:
#wget is a command line utility that allows us to download files from the internet to our local directory
!wget "https://raw.githubusercontent.com/zacharski/ml-class/master/data/bach.zip"

--2022-10-21 22:40:50--  https://raw.githubusercontent.com/zacharski/ml-class/master/data/bach.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41761 (41K) [application/zip]
Saving to: ‘bach.zip’

bach.zip            100%[===================>]  40.78K  --.-KB/s    in 0.01s   

2022-10-21 22:40:50 (3.99 MB/s) - ‘bach.zip’ saved [41761/41761]



In [128]:
#unzipping all the contents of the recently imported zip file
from zipfile import ZipFile
file = 'bach.zip'

with ZipFile(file, 'r') as zip:
  zip.extractall()

In [129]:
#importing the csv file into a dataframe
import pandas as pd
bach = pd.read_csv('bach.csv')
bach = bach.drop('choral_ID', axis=1)
bach = bach.drop('event_number', axis=1)
bach

,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
0,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
1,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,5,C_M
2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,2,C_M
3,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,G,4,G_m
5661,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,G,3,G_m
5662,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C,5,C_M
5663,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,C,3,C_M


# Data Wrangling

Before splitting up the data into training and testing sets we are going to need to make sure the features are usable, i.e encode them.
The columns that need to be encoded are all the notes columns and the bass column.

In [130]:
#first with the notes
notes = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B',]
for note in notes:
    bach[note] =  bach[note].apply(lambda x: 1 if x == 'YES' else 0)
bach

,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
0,1,0,0,0,0,1,0,0,0,1,0,0,F,3,F_M
1,1,0,0,0,1,0,0,1,0,0,0,0,E,5,C_M
2,1,0,0,0,1,0,0,1,0,0,0,0,E,2,C_M
3,1,0,0,0,0,1,0,0,0,1,0,0,F,3,F_M
4,1,0,0,0,0,1,0,0,0,1,0,0,F,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,0,0,1,0,0,0,0,1,0,0,1,0,G,4,G_m
5661,0,0,1,0,0,0,0,1,0,1,0,0,G,3,G_m
5662,1,0,0,0,1,0,0,1,0,0,0,0,C,5,C_M
5663,1,0,0,0,1,0,0,1,0,0,1,0,C,3,C_M


In [131]:
##lets change the notes columns to be formatted as "Note_thisnote"
for note in notes:
  newcolname = "Note_"+note
  bach = bach.rename(columns={note:newcolname})
bach

,Note_C,Note_C#,Note_D,Note_D#,Note_E,Note_F,Note_F#,Note_G,Note_G#,Note_A,Note_A#,Note_B,bass,meter,chord_label
0,1,0,0,0,0,1,0,0,0,1,0,0,F,3,F_M
1,1,0,0,0,1,0,0,1,0,0,0,0,E,5,C_M
2,1,0,0,0,1,0,0,1,0,0,0,0,E,2,C_M
3,1,0,0,0,0,1,0,0,0,1,0,0,F,3,F_M
4,1,0,0,0,0,1,0,0,0,1,0,0,F,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,0,0,1,0,0,0,0,1,0,0,1,0,G,4,G_m
5661,0,0,1,0,0,0,0,1,0,1,0,0,G,3,G_m
5662,1,0,0,0,1,0,0,1,0,0,0,0,C,5,C_M
5663,1,0,0,0,1,0,0,1,0,0,1,0,C,3,C_M


Next we have to onehot encode the bass column

In [132]:
bass = pd.get_dummies(bach['bass'])
cols = bass.columns.values.tolist()
for col in cols:
  newcolname = "Bass_"+col
  bass = bass.rename(columns={col:newcolname})
bass

,Bass_A,Bass_A#,Bass_Ab,Bass_B,Bass_Bb,Bass_C,Bass_C#,Bass_D,Bass_D#,Bass_Db,Bass_E,Bass_Eb,Bass_F,Bass_F#,Bass_G,Bass_G#
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
5661,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
5662,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
5663,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


Next we must remove the bass column from bach and add our new bass notes dataframe

In [133]:
bach = bach.drop('bass', axis=1)
bach = pd.concat([bach, bass], axis=1)
bach

,Note_C,Note_C#,Note_D,Note_D#,Note_E,Note_F,Note_F#,Note_G,Note_G#,Note_A,...,Bass_C#,Bass_D,Bass_D#,Bass_Db,Bass_E,Bass_Eb,Bass_F,Bass_F#,Bass_G,Bass_G#
0,1,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
1,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,1,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
5661,0,0,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
5662,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5663,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


Now we have our data in a more usable format

# Training and Basic Model Creation


Next we must split our data into training and testing data

In [134]:
bFeatures = bach.drop('chord_label', axis=1)
bLabels = bach['chord_label']
from sklearn.model_selection import train_test_split
bach_train_features, bach_test_features, bach_train_labels, bach_test_labels = train_test_split(bFeatures, bLabels, test_size = 0.2, random_state=42)

In [135]:
bFeatures

,Note_C,Note_C#,Note_D,Note_D#,Note_E,Note_F,Note_F#,Note_G,Note_G#,Note_A,...,Bass_C#,Bass_D,Bass_D#,Bass_Db,Bass_E,Bass_Eb,Bass_F,Bass_F#,Bass_G,Bass_G#
0,1,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
1,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,1,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
5661,0,0,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
5662,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5663,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [136]:
bLabels

0       F_M
1       C_M
2       C_M
3       F_M
4       F_M
       ... 
5660    G_m
5661    G_m
5662    C_M
5663    C_M
5664    F_M
Name: chord_label, Length: 5665, dtype: object

Let's see what a basic decision tree can do

In [137]:
#import decision tree module and fit the data
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(criterion='entropy')
clf.fit(bach_train_features, bach_train_labels)
predictions = clf.predict(bach_test_features)

from sklearn.metrics import accuracy_score

accuracy_score(predictions, bach_test_labels)

0.7184466019417476

and just out of curiosity

In [138]:
depth = clf.get_depth()
depth

15

So a basic decision tree created on this dataset has a depth of 15 and is around 71% accurate

# Bagging



Lets try a bagging classifier with 70 estimators, .7 max samples, .7 max features and no replacement

In [139]:
from sklearn.ensemble import BaggingClassifier
baseclf = DecisionTreeClassifier(criterion = 'entropy')
ensemble = BaggingClassifier(baseclf, n_estimators = 70, max_samples = .7,bootstrap = True, n_jobs=-1)
ensemble.fit(bach_train_features, bach_train_labels)
predictions = ensemble.predict(bach_test_features)
accuracy_score(predictions, bach_test_labels)

0.7308031774051191

# Pasting


In [140]:
clf = DecisionTreeClassifier(criterion='entropy')
pasting_clf = BaggingClassifier(clf, n_estimators=20, max_samples=.5, 
                                bootstrap=False, n_jobs=-1)
pasting_clf.fit(bach_train_features, bach_train_labels)
predictions = pasting_clf.predict(bach_test_features)
accuracy_score(bach_test_labels, predictions)

0.733451015004413

# Random Subspaces


In [141]:
clf = DecisionTreeClassifier(criterion='entropy')
subspace_clf = BaggingClassifier(clf, n_estimators=70, max_features=.7, bootstrap=True, n_jobs=-1)
subspace_clf.fit(bach_train_features, bach_train_labels)
predictions = subspace_clf.predict(bach_test_features)
accuracy_score(bach_test_labels, predictions)

0.7466902030008826

# Random Patches

In [142]:
clf= DecisionTreeClassifier(criterion='entropy', )
patches_clf = BaggingClassifier(clf, n_estimators=70, max_features=.7, max_samples =.7, bootstrap=False, n_jobs=-1)
patches_clf.fit(bach_train_features, bach_train_labels)
predictions=patches_clf.predict(bach_test_features)
accuracy_score(bach_test_labels, predictions)

0.7413945278022948

# XGBoost

Given that we just learned about XGBoost and that theoretically should be the most accurate, lets give it a shot first.
Our baseline will be 100 estimators

In [143]:
# from xgboost import XGBClassifier
# params = {'tree_method':'gpu_hist', 'predictor':'gpu_predictor'}
# model = XGBClassifier()
# model

In [144]:
# param_grid= {
#     'n_estimators': [50, 100, 150, 200],
# }

In [145]:
# from sklearn.model_selection import StratifiedKFold
# from sklearn.model_selection import RandomizedSearchCV
# param_comb = 5
# folds=5
# skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)
# random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=param_comb,  n_jobs=-1, 
#                                    cv=skf.split(bach_train_features,bach_train_labels), verbose=3)

In [146]:
# grid_result = random_search.fit(bach_train_features, bach_train_labels)

In [147]:
# predictions = grid_result.best_estimator_.predict(bach_test_features)
# accuracy_score(bach_test_labels, predictions)

My intial run was 73%. Not much better than our base decision tree.
Let's use some different hyperparameters


# Choosing the ideal classifer

The best perform classifier seems to be random subspaces at 74%. I tried many hyperparameters, but not sure what else I could have done to make the classifier perform better